In [36]:
import os
import torch
import pandas as pd
from skimage import io, transform
import numpy as np
import matplotlib.pyplot as plt
from torch.utils.data import Dataset, DataLoader
from torchvision import transforms, utils
os.environ["KMP_DUPLICATE_LIB_OK"]="TRUE"

class MuseumDataset(torch.utils.data.Dataset):
    """Датасет с фотками музея."""
    def __init__(self, csv_file, root_dir, transform=None):
        super(MuseumDataset, self).__init__()
        self.museum_items = pd.read_csv(csv_file, sep = ';', encoding = 'utf-8')
        self.root_dir = root_dir
        self.transform = transform
        
    def __getitem__(self, idx):
        if torch.is_tensor(idx):
            idx = idx.tolist()

        img_file = os.path.join(self.root_dir, 
                                str(self.museum_items.iloc[idx, 0]), self.museum_items.iloc[idx, 4])

        #print(img_file)
        image = io.imread(img_file)
        items = self.museum_items.iloc[idx, 1:]
        items = np.array([items], dtype=float).reshape(-1, 2)
        sample = {'image': image, 'items': items}

        if self.transform:
            sample = self.transform(sample)

        return sample


    def __len__(self):
        return len(self.museum_items)

In [37]:
from transformers import get_scheduler
from torch.optim import AdamW
from tqdm.auto import tqdm
import torch
from torch.utils.data import DataLoader
from transformers import ViTForImageClassification
from transformers import Trainer, TrainingArguments
from transformers import VisionEncoderDecoderModel, ViTFeatureExtractor, AutoTokenizer
import torch
from PIL import Image

df_train = MuseumDataset("dataset\\train.csv", os.path.join(os.getcwd(), "dataset", "train"))
model = ViTForImageClassification.from_pretrained('google/vit-base-patch16-224-in21k')
train_dataloader = DataLoader(df_train, batch_size=8,
                        shuffle=True, num_workers=0)
device = torch.device("cuda") if torch.cuda.is_available() else torch.device("cpu")
model.to(device)
num_epochs = 3
optimizer = AdamW(model.parameters(), lr=5e-5)
num_training_steps = num_epochs * len(train_dataloader)
lr_scheduler = get_scheduler(
    name="linear", optimizer=optimizer, num_warmup_steps=0, num_training_steps=num_training_steps
)

progress_bar = tqdm(range(num_training_steps))

model.train()
for epoch in range(num_epochs):
    for batch in train_dataloader:
        batch = {k: v.to(device) for k, v in batch.items()}
        outputs = model(**batch)
        loss = outputs.loss
        loss.backward()
        optimizer.step()
        lr_scheduler.step()
        optimizer.zero_grad()
        progress_bar.update(1)

Some weights of ViTForImageClassification were not initialized from the model checkpoint at google/vit-base-patch16-224-in21k and are newly initialized: ['classifier.bias', 'classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.
  0%|          | 0/7632 [00:51<?, ?it/s]


ValueError: could not convert string to float: 'Штык-нож'